In [2564]:
import pandas as pd

url = "https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv"

df = pd.read_csv(url, index_col = 0)
"""
1. 지역에 따라서 선호하는 게임 장르가 다를까 라는 질문에 대답을 하셔야합니다.

2. 연도별 게임의 트렌드가 있을까 라는 질문에 대답을 하셔야합니다.

3. 출고량이 높은 게임에 대한 분석 및 시각화 프로세스가 포함되어야 합니다.
"""   ## ++ . or 특정장르가 꾸준히 인기. or Year에 따라 변화하는 추이에 따라 (Genre, Platform) 설정.  // >> ~지역에서는 ~장르가 인기가 많다. 을 집중적으로 공략해야 한다
##근데 사실 이해하기에는 그림(plot)으로 표현하는게 최곤데.(https://eda-ai-lab.tistory.com/13) : 이상치 확인 EDA
df

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04
...,...,...,...,...,...,...,...,...,...
16594,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0,0.01
16595,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0,0,0
16596,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16597,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03


In [2565]:
# df.EU_Sales.unique()    ##column 값들 대충보기.
#df.info()   ## null data는 불완전 하므로 다 지우기. ('0'이랑 다름. - n133)   
##but Year는 명확하지 않지만 판매량이 엄청난 게임회사가 있을 수도. 확인 필요.Genre도 마찬가지. 진짜 버려도 되는 데이터인지 설멸할 수 있어야 신뢰도 증가.

# pd.DataFrame(df.isnull().sum(), columns=["결측치 개수"])

In [2566]:
df = df.dropna(axis=0)
# df.info()

In [2567]:
## K->M으로 단위변환 EDA
dfs = df.iloc[:,5:]   ## 나라별 sales

def toMil(data) :         ##시간날때 df['~'] = df['~'].apply(str)로 더간단화하기.
  if type(data) == str :
    if 'K' in data :
      data= data.replace('K','')
      data = float(data)*10**(-3)
    elif 'M' in data :
      data = data.replace('M','')
    else : data = data      ## 숫자형 data와 str형 data가 섞여있나..? (str제거 한다음 dtype 확인해보면 정확.) - 별로안중요.
  return data

for i in range(len(dfs.columns)):
  dfs.iloc[:,i] = dfs.iloc[:,i].apply(toMil)
  dfs.iloc[:,i] = dfs.iloc[:,i].apply(pd.to_numeric)

# dfs

In [2568]:
df1 = df.iloc[:,0:4]
df1
df = pd.concat([df1,dfs],axis=1)    ## 합치기, merge로도 해보기.

dfg = df.groupby('Genre')[['NA_Sales','EU_Sales','JP_Sales','Other_Sales']].sum()

# dfg    ##value를 수치화해서 % barplot subplot으로 큰 순서대로 그려넣으면 좋을듯. ##약간의 공통된 경향이 있긴 한데 그걸 어떻게 수치화해서 보여줄지.

In [2569]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

colors = sns.color_palette('colorblind',len(dfg.index)) ## Genre 색상 지정
dfg_sort = dfg.sort_values(by=dfg.columns[0], ascending=False)
dfg_sort['color'] = colors   ##Genre - color 지정.

# for i in range(len(dfg.columns)):         ## 각 지역별 genre 순위 plot으로 그리기.
#   fig, ax = plt.subplots(figsize=(13,8))      ## figsize
#   ax.set_title(dfg.columns[i])
#   dfNA = dfg_sort.sort_values(by=dfg.columns[i], ascending=False)     ##dfNA는 지역변수, dfEU, dfJP, dfOther...
#   ax.bar(dfNA.index, dfNA[dfg.columns[i]], color=dfNA.color)
## Sales의 총합은 Action이 제일 많을 수도 있으나, game수로 나눌 경우 평균의 판매량은 다를 수 있다.
## 라고 생각했으나, 그냥 인기가 제일 많다.로 단순히 생각하는게 나을듯. 왜냐하면 아직은 판매수익까지 예측하는게 아니니까.

##bottom 위에 쌓아서 합친 bar plot 필요.

In [2570]:
##1. 지역에 따라서 선호하는 게임 장르가 다를까 라는 질문에 대답을 하셔야합니다.
##가설 세우고 test, pvalue 검증. (t-test, chi-test, f-test)
##귀무가설 : 지역에 따라서 선호하는 게임 장르는 같을 것이다.
##scaling 후 각 row값들이 일치하는지 확인. 신뢰구간 ~범위내
## -> scaling하면 안됨. 바로 chi-2적용.

## 각 값에 대해서 pvalue, 인기순위에 대해서 pvalue
from scipy import stats
from scipy.stats import chi2_contingency

chi2_contingency(dfg, correction = False)   ## p value = 2.5e-125 < 0.05 : 변수간 연관이 있다. 귀무가설 채택.(대립가설을 기각.)
## 대체 귀무가설 대립가설은 어떻게 세우는거야. 뭐 다 얘기가 다르노...

(684.336340516923,
 9.635373883711956e-123,
 33,
 array([[842.66837143, 468.29974503, 248.7462714 , 153.26561214],
        [115.12174823,  63.97710792,  33.98265154,  20.93849231],
        [218.19118114, 121.25633043,  64.40759449,  39.68489393],
        [383.55878598, 213.15678595, 113.22226051,  69.76216756],
        [407.31414544, 226.35845479, 120.23457673,  74.08282303],
        [118.42260683,  65.81150838,  34.95702805,  21.53885674],
        [355.44491575, 197.53294304, 104.92336069,  64.64878052],
        [443.48093003, 246.45757869, 130.91060674,  80.66088454],
        [504.10653576, 280.14930922, 148.80660697,  91.68754804],
        [191.49899218, 106.42256462,  56.52835908,  34.83008412],
        [642.13752173, 356.85786709, 189.55180907, 116.79280211],
        [ 85.09426549,  47.28980484,  25.11887472,  15.47705495]]))

In [2571]:
# from sklearn.preprocessing import StandardScaler
# dfg_scale = StandardScaler().fit_transform(dfg)
# dfg_scale = pd.DataFrame(dfg_scale, index = dfg.index, columns=dfg.columns)      ## array->DF
# dfg_scale

### !! chi2 에서 pvalue가 0.05보다 작다는건 column간 연관이 있다는 것이고, scale안하고 바로 chi2 해야 된다.
# from sklearn.preprocessing import MinMaxScaler    ## MinMaxScale(0~1)적용 후 chi2-test하면 무조건 0.99~1나온다. 즉, 잘못된 방법.
# scaler = MinMaxScaler()
# dfg_MmScale = dfg.copy()
# dfg_MmScale[:] = scaler.fit_transform(dfg_MmScale[:])
# dfg_MmScale
# x = np.array(dfg_MmScale['Other_Sales'])[::-1]
# y = np.array(dfg_MmScale['NA_Sales'])[::-1]
# z = np.array(dfg_MmScale['EU_Sales'])[::-1]
# dfg_MmScale['test'] = x
# dfg_MmScale['test2'] = y
# dfg_MmScale['test3'] = z
# dfg_MmScale

In [2572]:

##시간되면 cov, corcoeff도 확인해보기.
# res4 = np.cov(dfg['NA_Sales'],dfg['EU_Sales'])[0,1]
# res4
# res = np.corrcoef(dfg['NA_Sales'],dfg['EU_Sales'])[0,1]
# res

## t-test 정규화한다음 2sample t-test하면 평균비교하는거라 같을;(pvalue=1)일 수밖에 없네
# stats.ttest_ind(dfg_scale['Other_Sales'][:],dfg_scale['test'][:])
# stats.ttest_ind(dfg['NA_Sales'][:],dfg['EU_Sales'][:])

# from stats import f_oneway
# g1 = dfg['NA_Sales']
# g2= dfg['EU_Sales']
# g3 = dfg['test']
# f_oneway(g1, g2, g3)


# dfg_scaleNA = dfg_scale.sort_values(by=dfg_scale.columns[0], ascending=False)
# dfg_scaleNA = dfg_scaleNA[0]

# dfg_scaleEU = dfg_scale.sort_values(by=dfg_scale.columns[1], ascending=False)
# dfg_scaleEU = 
### chi2 검증 기댓값을 plt로 표현하면 좋을 듯.
### https://www.jmp.com/ko_kr/statistics-knowledge-portal/chi-square-test/chi-square-test-of-independence.html

In [2573]:
## Year EDA
print(df.Year.unique())    ## 발표자료에 넣기. ~한 값들이 있더라. 네모로 해서. 전처리가 필요하다 의 근거. (96, 9, 6, 16,,,->19__ or 20__)

def toYear(y) :
  if y < 10 :
    y = str(y)
    y = '200'+y
  elif (y>=10) & (y<50):
    y = str(y)
    y = '20'+y
  elif (y>=50) & (y<1000):
    y = str(y)
    y = '19'+y
  else : y = y
  y=float(y)
  y=int(y)
  return y

df['Year'] = df['Year'].apply(toYear)
# df['Year'].unique()     ## 확인.
df
# dfy = df.groupby('Year')[['NA_Sales','EU_Sales','JP_Sales','Other_Sales']].sum()
# dfy     ## year별로 각나라 전체 sales 추이 line 그래프로.

[2.008e+03 2.009e+03 2.010e+03 2.005e+03 2.011e+03 2.007e+03 2.001e+03
 2.003e+03 2.006e+03 2.014e+03 2.015e+03 2.002e+03 1.997e+03 2.013e+03
 1.996e+03 2.004e+03 2.000e+03 1.984e+03 1.998e+03 2.016e+03 1.985e+03
 1.999e+03 9.000e+00 9.700e+01 1.995e+03 1.993e+03 2.012e+03 1.987e+03
 1.982e+03 1.100e+01 1.994e+03 1.990e+03 1.500e+01 1.992e+03 1.991e+03
 1.983e+03 1.988e+03 1.981e+03 3.000e+00 1.989e+03 9.600e+01 6.000e+00
 8.000e+00 1.986e+03 1.000e+00 5.000e+00 4.000e+00 1.000e+01 9.800e+01
 7.000e+00 1.600e+01 8.600e+01 1.400e+01 9.500e+01 2.017e+03 1.980e+03
 2.020e+03 2.000e+00 1.300e+01 0.000e+00 1.200e+01 9.400e+01]


,Name,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,Candace Kane's Candy Factory,DS,2008,Action,0.04,0.00,0.00,0.00
2,The Munchables,Wii,2009,Action,0.17,0.00,0.00,0.01
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010,Adventure,0.00,0.00,0.02,0.00
4,Deal or No Deal: Special Edition,DS,2010,Misc,0.04,0.00,0.00,0.00
5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010,Platform,0.12,0.09,0.00,0.04
...,...,...,...,...,...,...,...,...
16594,Ice Age 2: The Meltdown,GC,2006,Platform,0.15,0.04,0.00,0.01
16595,Rainbow Islands: Revolution,PSP,2005,Action,0.01,0.00,0.00,0.00
16596,NBA 2K16,PS3,2015,Sports,0.44,0.19,0.03,0.13
16597,Toukiden: The Age of Demons,PSV,2013,Action,0.05,0.05,0.25,0.03


In [2574]:
##2. 연도별 게임의 트렌드가 있을까 라는 질문에 대답을 하셔야합니다. ##각나라에서 & 전체.

# for i in range(df['Year'].min(axis=0),2000):     ##, df['Year'].max(axis=0)
#   mask = df['Year']==i
#   dfy = df[mask].groupby('Genre')[['NA_Sales','EU_Sales','JP_Sales','Other_Sales']].sum()
#   dfy['ThisYear'] = dfy.sum(axis=1)

#   dfy = dfy.merge(dfg_sort['color'], how='inner', on='Genre')     ## 전에 만들어놓은 color 가져다 쓰기.
#   dfy = dfy.sort_values(by=dfy.columns[4], ascending=True)    ## dfy.columns[4] = ThisYear
#   fig, ax = plt.subplots(figsize=(13,8))      ## figsize
#   ax.set_title(i)
#   bars = ax.barh(dfy.index, dfy['ThisYear'], color=dfy.color)    ## bar 끝부분에 Genre랑 값 넣기.

#   for bar in bars:      ## barh Text넣기.
#     width = bar.get_width()
#     label_y_pos = bar.get_y() + bar.get_height() / 2
#     ax.text(width, label_y_pos, s=width.round(2), va='center')
#   ##보조선
#   ax.set_axisbelow(True)
#   ax.xaxis.grid(True, color='gray', linestyle='dashed', linewidth=0.5)



# for container in ax.containers:
#     ax.bar_label(container)

# for i, v in enumerate(dfy['ThisYear']):
#   plt.text(v, i, fontsize=15, va='center')


#유튜브 영상넣기 (주소창 뒷부분에 v= '~' 부분 붙여넣기 ex)OtnSWvELXGO  )
# from IPython.display import YouTubeVideo, display 
# video = YouTubeVideo("OtnSWvELXG0", width=500) 
# display(video)

In [2575]:
##동영상으로. github 코드이용.
dfy_cross=df.copy()
dfy_cross = dfy_cross.set_index(df['Name'])
dff = dfy.iloc[:,4:8]
dfy_cross['Sales_Sum'] = dff.sum(axis=1)

dfy_cross = dfy_cross[['Year','Genre', 'Sales_Sum']]
dfy_cross
dfy_cross=dfy_cross.pivot(index=dfy_cross.Year, columns=dfy_cross.Genre, values=dfy_cross.Sales_Sum)

# dfy_cross.to_csv('dfy_cross.csv')


KeyError: ignored

In [ ]:
# ## 출고량이 제일 높은 게임에 대한 분석  ##1. genre, 2.Name
# # dfg['sum'] = dfg[,:].sum()
# dfg['sum'] = dfg.sum(axis=1)
# dfg = dfg.sort_values(by='sum', ascending=False)
# dfg     ##전체 출고량을 보면 Action > Sports > Shooter가 인기 있다. (%, 분산에 따라..?)

In [ ]:
# ## 역대 가장 인기 있는 game
# dfs['sum'] = dfs.sum(axis=1)

# df2 = pd.concat([df1,dfs],axis=1)    ## df1 : sales제외한 df의 앞 column부분,   dfs : df의 sales부분
# df2 = df2.sort_values(by='sum' ,ascending=False)      ##intro로 이렇게 격차가 크다(출시게임을 설계하는게 중요하다 & 관심끌기)는 걸 보여주면 좋을듯.(++ 특정지역에서만 성공? or 한지역에서의 성공이 다른 지역에서도 성공을 보장하는지. 관련성파악하기.)
# df2[0:10]